In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from boto3.session import Session
from io import StringIO
import random, boto3, pprint, datetime, sagemaker, time, ast
import timeit

In [4]:
bucketName = 'testsmdata2'
prefix = 'DERTestData/'

In [5]:
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
role

'arn:aws:iam::440616111601:role/DER-Control-Stack-ExecutionRole-14B0CCS9IEEI0'

In [6]:
def write_df_to_csv(bkt, prefix, f_name, df):

    """
    Writes the results on s3
    """

    path='{}{}'.format(prefix,f_name)
    print(f'{path}\n')

    csv_buffer = StringIO()
    df.to_csv(csv_buffer, index=False)

    s3_resource = boto3.client('s3')
    s3_resource.put_object(Body=csv_buffer.getvalue(), Bucket=bkt,
                           Key=path)

In [21]:
def readStreamData():
    client = boto3.client('kinesis')

    shard_id = 'shardId-000000000000' #we only have one shard!
    #shard_it = client.get_shard_iterator(StreamName="representativeDERResultStream", ShardId=shard_id, ShardIteratorType="LATEST")["ShardIterator"]
    ###shard_it = client.get_shard_iterator(StreamName="Veda5sMessage", ShardId=shard_id, ShardIteratorType="LATEST")["ShardIterator"]
    shard_it = client.get_shard_iterator(StreamName="DER-Kinesis-Stack-OutputKinesisStream-MQZDMZKEQBW1", ShardId=shard_id, ShardIteratorType="LATEST")["ShardIterator"]


    print('Waiting for data from DER ...')

    records = []
    while len(records) == 0:
        out = client.get_records(ShardIterator=shard_it, Limit=2)
        shard_it = out["NextShardIterator"]
        records = out['Records']
        #print (records);
        time.sleep(1)

    #print(records)
    recordDER = records[0]
    recordNetwork = records[1]
    #print('\n', recordDER, '\n', recordNetwork )
    print('Data is ready ...')
    return recordDER, recordNetwork

Waiting for data from DER ...
Data is ready ...


{'SequenceNumber': '49611517327250372450048355142530263661351397672737570818',
 'ApproximateArrivalTimestamp': datetime.datetime(2020, 10, 8, 6, 49, 6, 303000, tzinfo=tzlocal()),
 'Data': b'{"deviceId":"cfff2694180c","microgridId":"IN-DEMO-0001","eventTime":"2020-10-08 06:49:06.000","DERstd10Sec":null,"DERstd30Sec":null,"DERstd1Min":null,"DERstd2Min":null,"DERstd5Min":null,"Networkstd10Sec":0.0,"Networkstd30Sec":0.0,"Networkstd1Min":0.0,"Networkstd2Min":0.0,"Networkstd5Min":0.0,"DEREnergy10Sec":-0.0553718199849866,"networkEnergy10Sec":-9.99,"DEREnergy30Sec":-0.05288960976836997,"networkEnergy30Sec":-9.99,"DEREnergy1Min":-0.056664175482432036,"networkEnergy1Min":-9.99,"DEREnergy2Min":-0.07948555934609,"networkEnergy2Min":-9.99,"DEREnergy5Min":-0.07463396453536494,"networkEnergy5Min":-9.99}',
 'PartitionKey': '𨴄ᔍ⺑𨑒懠𐐺쳋𢆢芫↭郃𣞺𦣁𠀣𦳔뤁𥢣𢲲ሆ栰蔰뙭'}

In [30]:
def binary2Dict(record):

    byte_str = record['Data']
    dict_str = byte_str.decode("UTF-8")
    data = ast.literal_eval(dict_str)
    return data

binary2Dict(recordDER)

byte_str=b'{"deviceId":"cfff2694180c","microgridId":"IN-DEMO-0001","eventTime":"2020-10-08 06:49:06.000","DERstd10Sec":null,"DERstd30Sec":null,"DERstd1Min":null,"DERstd2Min":null,"DERstd5Min":null,"Networkstd10Sec":0.0,"Networkstd30Sec":0.0,"Networkstd1Min":0.0,"Networkstd2Min":0.0,"Networkstd5Min":0.0,"DEREnergy10Sec":-0.0553718199849866,"networkEnergy10Sec":-9.99,"DEREnergy30Sec":-0.05288960976836997,"networkEnergy30Sec":-9.99,"DEREnergy1Min":-0.056664175482432036,"networkEnergy1Min":-9.99,"DEREnergy2Min":-0.07948555934609,"networkEnergy2Min":-9.99,"DEREnergy5Min":-0.07463396453536494,"networkEnergy5Min":-9.99}'
dict_str={"deviceId":"cfff2694180c","microgridId":"IN-DEMO-0001","eventTime":"2020-10-08 06:49:06.000","DERstd10Sec":null,"DERstd30Sec":null,"DERstd1Min":null,"DERstd2Min":null,"DERstd5Min":null,"Networkstd10Sec":0.0,"Networkstd30Sec":0.0,"Networkstd1Min":0.0,"Networkstd2Min":0.0,"Networkstd5Min":0.0,"DEREnergy10Sec":-0.0553718199849866,"networkEnergy10Sec":-9.99,"DEREnergy30

ValueError: malformed node or string: <_ast.Name object at 0x7fdfafdc5ba8>

In [33]:
#webhook_data = {"side": "BUY","key": "8234023409fa3242309sdfasdf903024917325"}
webhook_data = {"deviceId":"cfff2694180c","microgridId":"IN-DEMO-0001",
                "eventTime":"2020-10-08 06:49:06.000","DERstd10Sec":null,"DERstd30Sec":null,
                "DERstd1Min":null,"DERstd2Min":null,"DERstd5Min":null,"Networkstd10Sec":0.0,
                "Networkstd30Sec":0.0,"Networkstd1Min":0.0,"Networkstd2Min":0.0,"Networkstd5Min":0.0,
                "DEREnergy10Sec":-0.0553718199849866,"networkEnergy10Sec":-9.99,
                "DEREnergy30Sec":-0.05288960976836997,"networkEnergy30Sec":-9.99,
                "DEREnergy1Min":-0.056664175482432036,"networkEnergy1Min":-9.99,
                "DEREnergy2Min":-0.07948555934609,"networkEnergy2Min":-9.99,
                "DEREnergy5Min":-0.07463396453536494,"networkEnergy5Min":-9.99}

def parse_webhook(webhook_data):

    """
    :param webhook_data: POST data from tradingview, as a string.
    :return: Dictionary version of string.
    """

    data = ast.literal_eval(str(webhook_data)) # this fixes the issue
    return data
print(parse_webhook(webhook_data))
print(type(str(webhook_data)))

NameError: name 'null' is not defined

In [9]:
def filterRecords(Networkdata, DERdata):
    Networkrecords = {}
    for key in Networkdata.keys():
        #print(key)
        Networkrecords[key] = Networkdata[key]
    #print(Networkrecords)    

    Networkrecords.pop('DERstd10Sec', None)
    Networkrecords.pop('DERstd30Sec', None)
    Networkrecords.pop('DERstd1Min', None)
    Networkrecords.pop('DERstd2Min', None)
    Networkrecords.pop('DERstd5Min', None)

    Networkrecords.pop('DEREnergy10Sec', None)
    Networkrecords.pop('DEREnergy30Sec', None)
    Networkrecords.pop('DEREnergy1Min', None)
    Networkrecords.pop('DEREnergy2Min', None)
    Networkrecords.pop('DEREnergy5Min', None)
    #print(Networkrecords)
    
    DERrecords = {}
    for key in DERdata.keys():
        #print(key)
        DERrecords[key] = DERdata[key]
    #print(DERrecords)    
    #records['TimeStamp'] = records['ApproximateArrivalTimestamp']

    DERrecords.pop('Networkstd10Sec', None)
    DERrecords.pop('Networkstd30Sec', None)
    DERrecords.pop('Networkstd1Min', None)
    DERrecords.pop('Networkstd2Min', None)
    DERrecords.pop('Networkstd5Min', None)

    DERrecords.pop('networkEnergy10Sec', None)
    DERrecords.pop('networkEnergy30Sec', None)
    DERrecords.pop('networkEnergy1Min', None)
    DERrecords.pop('networkEnergy2Min', None)
    DERrecords.pop('networkEnergy5Min', None)
    #print(DERrecords)

    records = {}
    if DERrecords['eventTime'] == Networkrecords['eventTime']:
        for key in DERrecords.keys():
            records[key] = DERrecords[key]

        for key in Networkrecords.keys():
            records[key] = Networkrecords[key]
    
    return records

#records = filterRecords(Networkdata, DERdata)
#records

In [10]:
def finalDataFrame(records, s, t):
    df = pd.DataFrame(records, index=[0])
    df ['state'] = s
    df ['duration'] = t[0]
    for timespan in ['10Sec', '30Sec', '1Min', '2Min', '5Min']:
    #if (df['networkEnergy' + timespan])!=0:
        df['pen' + timespan] = (1.5 * df['DEREnergy' + timespan]) / ((1.5 * df['DEREnergy' + timespan]) + df['networkEnergy' + timespan])
    
    for timespan in ['10Sec', '30Sec', '1Min', '2Min', '5Min']:
        df = df.drop (columns = 'DEREnergy' + timespan)
        df = df.drop (columns = 'networkEnergy' + timespan)
        
    df['eventTime'] = pd.to_datetime(df['eventTime']) + datetime.timedelta(hours = 5, minutes =30)
    df['eventTime'] = df['eventTime'].apply(lambda x: x.strftime('%Y%m%dT%H%M%SZ')).astype(str)
    
    return df

In [14]:
N = 100

durationTimes = [10, 20, 30, 60, 120]
States = [[1, 1, 1, 1], 
          [0, 1, 1, 1],
          [0, 0, 1, 1], 
          [0, 0, 0, 1]
         ]

In [15]:
duration = random.sample(durationTimes, 1)
state = random.sample(States, 1)

state , duration

([[0, 1, 1, 1]], [120])

In [16]:
start = timeit.default_timer() 

state , duration = [[1, 1, 1, 1]] , [0]
i = 0
dfAll = pd.DataFrame()

for i in range (N):
    
    dfStage = pd.DataFrame()
    i += 1
    print (f'Sample Number {i} :')

    try: 
        
        print (f'State before sampling: {state} , {duration}')
        recordDER, recordNetwork = readStreamData()
    
        DERdata, Networkdata = binary2Dict(recordDER), binary2Dict(recordNetwork)
        records = filterRecords(Networkdata, DERdata)
        dfBefore = finalDataFrame(records, state, duration)
        sumStatesBefore = np.array(state).sum()
        
        state = random.sample(States, 1)
        duration = random.sample(durationTimes, 1)
        sumStatesAfter = np.array(state).sum()
        
        print (f'State after sampling: {state} , {duration}')
        print ('Sending Command to DER ...')

        # {Call the lambda function to send MQTT message to DER}
        
        if sumStatesAfter > sumStatesBefore:
            time.sleep(180.0 / 1.0)    # 180 Ramp Up Time (Ping) 
        
        time.sleep(int(duration[0])/1.0)  # duration[0] Wait Time 

        recordDER, recordNetwork = readStreamData()
        
        DERdata, Networkdata = binary2Dict(recordDER), binary2Dict(recordNetwork)
        records = filterRecords(Networkdata, DERdata)
        dfAfter = finalDataFrame(records, state , duration)


        dfStage = pd.concat([dfBefore, dfAfter], axis=0)
        fileName = 'DER_testStreamTransaction-' + dfAfter['eventTime'][0] + '.csv'
        write_df_to_csv(bucketName, prefix, fileName, dfStage)
        
        dfAll = pd.concat([dfAll, dfStage], axis=0)
        
    except Exception as e: 
        
        print (getattr(e, 'message', repr(e)))
        continue 

fileName = 'DER_testStreamWholeExperimentTransaction-' + dfAfter['eventTime'][0] + '.csv'
write_df_to_csv(bucketName, prefix, fileName, dfAll.reset_index())

stop = timeit.default_timer()
print('Running Time: ', stop - start)

Sample Number 1 :
State before sampling: [[1, 1, 1, 1]] , [0]
Waiting for data from DER ...
Data is ready ...
ValueError('malformed node or string: <_ast.Name object at 0x7fdfafeb8a90>',)
Sample Number 2 :
State before sampling: [[1, 1, 1, 1]] , [0]
Waiting for data from DER ...
Data is ready ...
ValueError('malformed node or string: <_ast.Name object at 0x7fdfafe57f60>',)
Sample Number 3 :
State before sampling: [[1, 1, 1, 1]] , [0]
Waiting for data from DER ...
Data is ready ...
ValueError('malformed node or string: <_ast.Name object at 0x7fdfafe6a4e0>',)
Sample Number 4 :
State before sampling: [[1, 1, 1, 1]] , [0]
Waiting for data from DER ...
Data is ready ...
ValueError('malformed node or string: <_ast.Name object at 0x7fdfafe7e128>',)
Sample Number 5 :
State before sampling: [[1, 1, 1, 1]] , [0]
Waiting for data from DER ...
Data is ready ...
ValueError('malformed node or string: <_ast.Name object at 0x7fdfafe0f0f0>',)
Sample Number 6 :
State before sampling: [[1, 1, 1, 1]] , [0

KeyboardInterrupt: 

In [76]:
dfAll.reset_index()#[['state', 'duration']]

,index


In [50]:
np.array(state).sum()

2

In [11]:
for i in range (N):
    
    dfAll = pd.DataFrame()
    
    s, t = [[0, 0, 0, 0]] , [0]
    try: 
        recordDER, recordNetwork = readStreamData()
    
        DERdata, Networkdata = binary2Dict(recordDER), binary2Dict(recordNetwork)
        records = filterRecords(Networkdata, DERdata)
        dfBefore = finalDataFrame(records, s, t)

        s = random.sample(States, 1)
        t = random.sample(T, 1)
        print (s, t)

        # {Call the lambda function to send MQTT message to DER}

        time.sleep(int(t[0]))

        recordDER, recordNetwork = readStreamData()
        
        DERdata, Networkdata = binary2Dict(recordDER), binary2Dict(recordNetwork)
        records = filterRecords(Networkdata, DERdata)
        dfAfter = finalDataFrame(records, s, t)


        dfAll = pd.concat([dfAfter, dfBefore], axis=0)
        fileName = 'derTest' + dfAfter['eventTime'][0] + '.csv'
        write_df_to_csv(bucketName, prefix, fileName, dfAll)
    
    except: continue 

Reading Stream ...
Data is ready ...
[[0, 0, 1, 1]] [20]
Reading Stream ...
Data is ready ...
DERTestData/derTest20200925T105813Z.csv
Reading Stream ...
Data is ready ...
[[0, 1, 1, 1]] [60]
Reading Stream ...
Data is ready ...
DERTestData/derTest20200925T105923Z.csv
Reading Stream ...
Data is ready ...
[[1, 1, 1, 1]] [60]
Reading Stream ...
Data is ready ...
DERTestData/derTest20200925T110033Z.csv


In [35]:
dfBefore

,deviceId,microgridId,eventTime,DERstd10Sec,DERstd30Sec,DERstd1Min,DERstd2Min,DERstd5Min,Networkstd10Sec,Networkstd30Sec,Networkstd1Min,Networkstd2Min,Networkstd5Min,s,t,pen10Sec,pen30Sec,pen1Min,pen2Min,pen5Min
0,demo-dss-001,IN-demo-dss-0001,20200923T053842Z,0.001537,0.003149,0.007903,0.015686,0.011388,0.001537,0.003149,0.007903,0.015686,0.011388,"[0, 0, 0, 1]",10,0.572479,0.587535,0.594541,0.596755,0.597759


In [36]:
dfAfter

,deviceId,microgridId,eventTime,DERstd10Sec,DERstd30Sec,DERstd1Min,DERstd2Min,DERstd5Min,Networkstd10Sec,Networkstd30Sec,Networkstd1Min,Networkstd2Min,Networkstd5Min,s,t,pen10Sec,pen30Sec,pen1Min,pen2Min,pen5Min
0,demo-dss-001,IN-demo-dss-0001,20200923T053707Z,0.003013,0.007422,0.006902,0.005623,0.006512,0.003013,0.007422,0.006902,0.005623,0.006512,"[0, 0, 1, 1]",30,0.621517,0.606078,0.602463,0.601036,0.600571


In [40]:
dfAll = pd.concat([dfAfter, dfBefore], axis=0)

dfAll

,deviceId,microgridId,eventTime,DERstd10Sec,DERstd30Sec,DERstd1Min,DERstd2Min,DERstd5Min,Networkstd10Sec,Networkstd30Sec,Networkstd1Min,Networkstd2Min,Networkstd5Min,s,t,pen10Sec,pen30Sec,pen1Min,pen2Min,pen5Min
0,demo-dss-001,IN-demo-dss-0001,20200923T053707Z,0.003013,0.007422,0.006902,0.005623,0.006512,0.003013,0.007422,0.006902,0.005623,0.006512,"[0, 0, 1, 1]",30,0.621517,0.606078,0.602463,0.601036,0.600571
0,demo-dss-001,IN-demo-dss-0001,20200923T053842Z,0.001537,0.003149,0.007903,0.015686,0.011388,0.001537,0.003149,0.007903,0.015686,0.011388,"[0, 0, 0, 1]",10,0.572479,0.587535,0.594541,0.596755,0.597759


In [8]:
client = boto3.client('kinesis')

shard_id = 'shardId-000000000000' #we only have one shard!
shard_it = client.get_shard_iterator(StreamName="representativeDERResultStream", ShardId=shard_id, ShardIteratorType="LATEST")["ShardIterator"]
#shard_it = client.get_shard_iterator(StreamName="Veda5sMessage", ShardId=shard_id, ShardIteratorType="LATEST")["ShardIterator"]
print('Reading Stream ...')

records = []
while len(records) == 0:
    out = client.get_records(ShardIterator=shard_it, Limit=2)
    shard_it = out["NextShardIterator"]
    records = out['Records']
    #print (records);
    time.sleep(1)

#print(records)
recordDER = records[0]
recordNetwork = records[1]
#print('\n', recordDER, '\n', recordNetwork )
print('Data is ready ...')

Reading Stream ...
Data is ready ...


In [9]:
import ast
byte_str = recordDER['Data']
dict_str = byte_str.decode("UTF-8")
DERdata = ast.literal_eval(dict_str)
result = repr(DERdata)
#print(result)

byte_str = recordNetwork['Data']
dict_str = byte_str.decode("UTF-8")
Networkdata = ast.literal_eval(dict_str)
result = repr(Networkdata)
#print(result)

In [10]:
DERdata

{'deviceId': 'demo-dss-001',
 'microgridId': 'IN-demo-dss-0001',
 'eventTime': '2020-09-22 23:41:19.000',
 'DERstd10Sec': 4.2546615021187965e-05,
 'DERstd30Sec': 0.00018553360699284947,
 'DERstd1Min': 0.003321962186453453,
 'DERstd2Min': 0.003321962186453453,
 'DERstd5Min': 0.003321962186453453,
 'Networkstd10Sec': 0.0,
 'Networkstd30Sec': 0.0,
 'Networkstd1Min': 0.0,
 'Networkstd2Min': 0.0,
 'Networkstd5Min': 0.0,
 'DEREnergy10Sec': -0.04198468666666635,
 'networkEnergy10Sec': -9.99,
 'DEREnergy30Sec': -0.05981654076923079,
 'networkEnergy30Sec': -9.99,
 'DEREnergy1Min': -0.07145516263157894,
 'networkEnergy1Min': -9.99,
 'DEREnergy2Min': -0.07145516263157894,
 'networkEnergy2Min': -9.99,
 'DEREnergy5Min': -0.07145516263157894,
 'networkEnergy5Min': -9.99}

In [11]:
Networkdata

{'deviceId': 'demo-dss-001',
 'microgridId': 'IN-demo-dss-0001',
 'eventTime': '2020-09-22 23:41:19.000',
 'DERstd10Sec': 0.0,
 'DERstd30Sec': 0.0,
 'DERstd1Min': 0.0,
 'DERstd2Min': 0.0,
 'DERstd5Min': 0.0,
 'Networkstd10Sec': 4.2546615021187965e-05,
 'Networkstd30Sec': 0.00018553360699284947,
 'Networkstd1Min': 0.003321962186453453,
 'Networkstd2Min': 0.003321962186453453,
 'Networkstd5Min': 0.003321962186453453,
 'DEREnergy10Sec': -9.99,
 'networkEnergy10Sec': -0.05379339749999976,
 'DEREnergy30Sec': -9.99,
 'networkEnergy30Sec': -0.06191675428571431,
 'DEREnergy1Min': -9.99,
 'networkEnergy1Min': -0.0717614448275862,
 'DEREnergy2Min': -9.99,
 'networkEnergy2Min': -0.0717614448275862,
 'DEREnergy5Min': -9.99,
 'networkEnergy5Min': -0.0717614448275862}

In [12]:
Networkrecords = {}
for key in Networkdata.keys():
    #print(key)
    Networkrecords[key] = Networkdata[key]
#print(Networkrecords)    

Networkrecords.pop('DERstd10Sec', None)
Networkrecords.pop('DERstd30Sec', None)
Networkrecords.pop('DERstd1Min', None)
Networkrecords.pop('DERstd2Min', None)
Networkrecords.pop('DERstd5Min', None)

Networkrecords.pop('DEREnergy10Sec', None)
Networkrecords.pop('DEREnergy30Sec', None)
Networkrecords.pop('DEREnergy1Min', None)
Networkrecords.pop('DEREnergy2Min', None)
Networkrecords.pop('DEREnergy5Min', None)
Networkrecords

{'deviceId': 'demo-dss-001',
 'microgridId': 'IN-demo-dss-0001',
 'eventTime': '2020-09-22 23:41:19.000',
 'Networkstd10Sec': 4.2546615021187965e-05,
 'Networkstd30Sec': 0.00018553360699284947,
 'Networkstd1Min': 0.003321962186453453,
 'Networkstd2Min': 0.003321962186453453,
 'Networkstd5Min': 0.003321962186453453,
 'networkEnergy10Sec': -0.05379339749999976,
 'networkEnergy30Sec': -0.06191675428571431,
 'networkEnergy1Min': -0.0717614448275862,
 'networkEnergy2Min': -0.0717614448275862,
 'networkEnergy5Min': -0.0717614448275862}

In [13]:
DERrecords = {}
for key in DERdata.keys():
    #print(key)
    DERrecords[key] = DERdata[key]
#print(DERrecords)    
#records['TimeStamp'] = records['ApproximateArrivalTimestamp']

DERrecords.pop('Networkstd10Sec', None)
DERrecords.pop('Networkstd30Sec', None)
DERrecords.pop('Networkstd1Min', None)
DERrecords.pop('Networkstd2Min', None)
DERrecords.pop('Networkstd5Min', None)

DERrecords.pop('networkEnergy10Sec', None)
DERrecords.pop('networkEnergy30Sec', None)
DERrecords.pop('networkEnergy1Min', None)
DERrecords.pop('networkEnergy2Min', None)
DERrecords.pop('networkEnergy5Min', None)
DERrecords

{'deviceId': 'demo-dss-001',
 'microgridId': 'IN-demo-dss-0001',
 'eventTime': '2020-09-22 23:41:19.000',
 'DERstd10Sec': 4.2546615021187965e-05,
 'DERstd30Sec': 0.00018553360699284947,
 'DERstd1Min': 0.003321962186453453,
 'DERstd2Min': 0.003321962186453453,
 'DERstd5Min': 0.003321962186453453,
 'DEREnergy10Sec': -0.04198468666666635,
 'DEREnergy30Sec': -0.05981654076923079,
 'DEREnergy1Min': -0.07145516263157894,
 'DEREnergy2Min': -0.07145516263157894,
 'DEREnergy5Min': -0.07145516263157894}

In [14]:
records = {}
if DERrecords['eventTime'] == Networkrecords['eventTime']:
    for key in DERrecords.keys():
        records[key] = DERrecords[key]

    for key in Networkrecords.keys():
        records[key] = Networkrecords[key]
        
#records

In [15]:
df = pd.DataFrame(records, index=[0])
df

,deviceId,microgridId,eventTime,DERstd10Sec,DERstd30Sec,DERstd1Min,DERstd2Min,DERstd5Min,DEREnergy10Sec,DEREnergy30Sec,...,Networkstd10Sec,Networkstd30Sec,Networkstd1Min,Networkstd2Min,Networkstd5Min,networkEnergy10Sec,networkEnergy30Sec,networkEnergy1Min,networkEnergy2Min,networkEnergy5Min
0,demo-dss-001,IN-demo-dss-0001,2020-09-22 23:41:19.000,0.000043,0.000186,0.003322,0.003322,0.003322,-0.041985,-0.059817,...,0.000043,0.000186,0.003322,0.003322,0.003322,-0.053793,-0.061917,-0.071761,-0.071761,-0.071761


In [16]:
for timespan in ['10Sec', '30Sec', '1Min', '2Min', '5Min']:
    #if (df['networkEnergy' + timespan])!=0:
        df['pen' + timespan] = df['DEREnergy' + timespan] / df['networkEnergy' + timespan]

In [17]:
for timespan in ['10Sec', '30Sec', '1Min', '2Min', '5Min']:
    df = df.drop (columns = 'DEREnergy' + timespan)
    df = df.drop (columns = 'networkEnergy' + timespan)
df

,deviceId,microgridId,eventTime,DERstd10Sec,DERstd30Sec,DERstd1Min,DERstd2Min,DERstd5Min,Networkstd10Sec,Networkstd30Sec,Networkstd1Min,Networkstd2Min,Networkstd5Min,pen10Sec,pen30Sec,pen1Min,pen2Min,pen5Min
0,demo-dss-001,IN-demo-dss-0001,2020-09-22 23:41:19.000,0.000043,0.000186,0.003322,0.003322,0.003322,0.000043,0.000186,0.003322,0.003322,0.003322,0.78048,0.96608,0.995732,0.995732,0.995732


In [18]:
df['eventTime'] = pd.to_datetime(df['eventTime']) + datetime.timedelta(hours = 5, minutes =30)
df['eventTime'] = df['eventTime'].apply(lambda x: x.strftime('%Y%m%dT%H%M%SZ')).astype(str)
df

,deviceId,microgridId,eventTime,DERstd10Sec,DERstd30Sec,DERstd1Min,DERstd2Min,DERstd5Min,Networkstd10Sec,Networkstd30Sec,Networkstd1Min,Networkstd2Min,Networkstd5Min,pen10Sec,pen30Sec,pen1Min,pen2Min,pen5Min
0,demo-dss-001,IN-demo-dss-0001,20200923T051119Z,0.000043,0.000186,0.003322,0.003322,0.003322,0.000043,0.000186,0.003322,0.003322,0.003322,0.78048,0.96608,0.995732,0.995732,0.995732


In [19]:
bucketName = 'testsmdata2'
prefix = 'DERTestData/'
fileName = 'derTest' + df['eventTime'][0] + '.csv'

write_df_to_csv(bucketName, prefix, fileName, df)

DERTestData/derTest20200923T051119Z.csv


In [122]:
df['eventTime'][0]

'20200921T155104Z'

In [24]:
client = boto3.client('kinesisanalytics')
client

In [25]:
response = client.get_analyzed_resource(
    analyzerArn='arn:aws:kinesisanalytics:ap-south-1:440616111601:application/derControl',
    resourceArn='arn:aws:kinesis:ap-south-1:440616111601:stream/Veda5sMessage'
)

AttributeError: 'KinesisAnalytics' object has no attribute 'get_analyzed_resource'

In [ ]:
add_application_output(**kwargs)

In [23]:
response = client.get_analyzed_resource(
    analyzerArn="arn:aws:kinesisanalytics:ap-south-1:440616111601:application/DSS_5sStatusMessageLowVoltage",
    resourceArn='arn:aws:kinesis:ap-south-1:440616111601:stream/DSS_5sStatusMessageLowVoltage'
)

ValidationException: An error occurred (ValidationException) when calling the GetAnalyzedResource operation: Invalid analyzerArn